### Model Composition ServerHandle APIs

© 2019-2022, Anyscale. All Rights Reserved

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
➡ [Next notebook](./ex_04_inference_graphs.ipynb) <br>
⬅️ [Previous notebook](./ex_02_ray_serve_fastapi.ipynb) <br>

<img src="images/PatternsMLProduction.png" width="70%" height="40%">

### Learning Objective:
In this tutorial, you will learn how to:

 * compose complex models using ServeHandle APIs
 * deploy each discreate model as a seperate model deployment
 * use a single class deployment to include individual as a single model composition
 * deploy and serve this singluar model composition


In this short tutorial we going to use HuggingFace Transformer 🤗 to accomplish three tasks:
 1. Analyse the sentiment of a tweet: Positive or Negative
 2. Translate it into French
 3. Demonstrate the model composition deployment pattern using ServeHandle APIs
 
 <img src="images/sentiment_analysis.jpeg" width="70%" height="40%">

In [1]:
from transformers import TranslationPipeline, TextClassificationPipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
import torch
import requests
from ray import serve

Package pickle5 becomes unnecessary in Python 3.8 and above. Its presence may confuse libraries including Ray. Please uninstall the package.


These are example 🐦 tweets, some made up, some extracted from a dog lover's twitter handle. In a real use case,
these could come live from a Tweeter handle using [Twitter APIs](https://developer.twitter.com/en/docs/twitter-api/getting-started/getting-access-to-the-twitter-api). 

In [2]:
TWEETS = ["Tonight on my walk, I got mad because mom wouldn't let me play with this dog. We stared at each other...he never blinked!",
          "Sometimes. when i am bored. i will stare at nothing. and try to convince the human. that there is a ghost",
          "You little dog shit, you peed and pooed on my new carpet. Bad dog!",
          "I would completely believe you. Dogs and little children - very innocent and open to seeing such things",
          "You've got too much time on your paws. Go check on the skittle. under the, fridge",
          "You sneaky little devil, I can't live without you!!!",
          "It's true what they say about dogs: they are you BEST BUDDY, no matter what!",
          "This dog is way dope, just can't enough of her",
          "This dog is way cool, just can't enough of her",
          "Is a dog really the best pet?",
          "Cats are better than dogs",
          "Totally dissastified with the dog. Worst dog ever",
          "Briliant dog! Reads my moods like a book. Senses my moods and reacts. What a companinon!"
          ]

Utiliy function to fetch a tweet; these could very well be live tweets coming from Twitter API for a user or a #hashtag

In [3]:
def fetch_tweet_text(i):
    text = TWEETS[i]
    return text

### Sentiment model deployment

Our function deployment model to analyse the tweet using a pretrained transformer from HuggingFace 🤗.
Note we have number of `replicas=1` but to scale it, we can increase the number of replicas, as
we have done below.

In [4]:
@serve.deployment(num_replicas=1)
class SentimentTweet:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english", model_max_length=128)
        self.model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
        self.pipeline = TextClassificationPipeline(model=self.model, tokenizer=self.tokenizer, task="sentiment-analysis")

    def sentiment(self, text: str):
        return (f"label:{self.pipeline(text)[0]['label']}; score={self.pipeline(text)[0]['score']}")

### Translation model deployment

Our function to translate a tweet from English --> French using a pretrained Transformer from HuggingFace 🤗

In [5]:
@serve.deployment(num_replicas=2)
class TranslateTweet:
    def __init__(self):
         self.tokenizer = AutoTokenizer.from_pretrained("t5-small", model_max_length=128)
         self.model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
         self.use_gpu = 0 if torch.cuda.is_available() else -1
         self.pipeline = TranslationPipeline(self.model, self.tokenizer, task="translation_en_to_fr", device=self.use_gpu)

    def translate(self, text: str):
        return self.pipeline(text)[0]['translation_text']

### Use the Model Composition pattern

<img src="images/tweet_composition.png" width="60%" height="25%">

A composed class is deployed with both sentiment analysis and translations models' ServeHandles initialized in the constructor.

In [6]:
@serve.deployment(route_prefix="/composed", num_replicas=2)
class ComposedModel:
    def __init__(self, translate, sentiment):
        # fetch and initialize deployment handles
        self.translate_model = translate
        self.sentiment_model = sentiment

    async def __call__(self, http_request):
        data = await http_request.json()
        sentiment_ref = await self.sentiment_model.sentiment.remote(data)
        trans_text_ref = await self.translate_model.translate.remote(data)
        sentiment_val = await sentiment_ref
        trans_text = await trans_text_ref

        return {'Sentiment': sentiment_val, 'Translated Text': trans_text}

### Deploy our models 

Deploy our models. Our composed class is deployed with both sentiment analysis and translations models' `ClassNode` initialized in the constructor.

In [7]:
translate_cls_node = TranslateTweet.bind()
sentiment_cls_node = SentimentTweet.bind()
compose_cls_node = ComposedModel.bind(translate_cls_node,sentiment_cls_node )

serve.run(compose_cls_node)

2022-08-09 09:38:11,565	INFO worker.py:1481 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265.
(ServeController pid=16150) INFO 2022-08-09 09:38:14,922 controller 16150 http_state.py:129 - Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-023b316d1dbebbf2bc7feec83e47d2bd90781c7c4e5c3d5481729d9a' on node '023b316d1dbebbf2bc7feec83e47d2bd90781c7c4e5c3d5481729d9a' listening on '127.0.0.1:8000'
(HTTPProxyActor pid=16160) INFO:     Started server process [16160]
(ServeController pid=16150) INFO 2022-08-09 09:38:16,360 controller 16150 deployment_state.py:1232 - Adding 2 replicas to deployment 'TranslateTweet'.
(ServeController pid=16150) INFO 2022-08-09 09:38:16,380 controller 16150 deployment_state.py:1232 - Adding 1 replicas to deployment 'SentimentTweet'.
(ServeController pid=16150) INFO 2022-08-09 09:38:16,392 controller 16150 deployment_state.py:1232 - Adding 2 replicas to deployment 'ComposedModel'.


RayServeSyncHandle(deployment='ComposedModel')

### Send HTTP requests to our deployment model

In [8]:
for i in range(len(TWEETS)):
    tweet = fetch_tweet_text(i)
    response = requests.post("http://127.0.0.1:8000/composed", json=tweet)
    print(f"tweet request... : {tweet}")
    print(f"tweet response:{response.text}")

(ServeReplica:SentimentTweet pid=16166) INFO 2022-08-09 09:38:37,682 SentimentTweet SentimentTweet#FBEAev replica.py:482 - HANDLE sentiment OK 110.6ms


tweet request... : Tonight on my walk, I got mad because mom wouldn't let me play with this dog. We stared at each other...he never blinked!
tweet response:{"Sentiment": "label:POSITIVE; score=0.9651218056678772", "Translated Text": "Ce soir, j'ai \u00e9t\u00e9 fou parce que ma m\u00e8re ne me laisse pas jouer avec ce chien."}


(HTTPProxyActor pid=16160) INFO 2022-08-09 09:38:39,146 http_proxy 127.0.0.1 http_proxy.py:315 - POST /composed 200 6660.4ms
(ServeReplica:TranslateTweet pid=16164) INFO 2022-08-09 09:38:39,144 TranslateTweet TranslateTweet#gIwrMB replica.py:482 - HANDLE translate OK 1565.2ms
(ServeReplica:ComposedModel pid=16167) INFO 2022-08-09 09:38:39,145 ComposedModel ComposedModel#KRrrri replica.py:482 - HANDLE __call__ OK 6655.0ms
(ServeReplica:SentimentTweet pid=16166) INFO 2022-08-09 09:38:43,408 SentimentTweet SentimentTweet#FBEAev replica.py:482 - HANDLE sentiment OK 97.7ms


tweet request... : Sometimes. when i am bored. i will stare at nothing. and try to convince the human. that there is a ghost
tweet response:{"Sentiment": "label:NEGATIVE; score=0.99788898229599", "Translated Text": "Parfois. quand j'ennuie. je ne regarderai rien. et essayerai de convaincre l'homme."}


(HTTPProxyActor pid=16160) INFO 2022-08-09 09:38:44,452 http_proxy 127.0.0.1 http_proxy.py:315 - POST /composed 200 5301.4ms
(ServeReplica:TranslateTweet pid=16164) INFO 2022-08-09 09:38:44,450 TranslateTweet TranslateTweet#gIwrMB replica.py:482 - HANDLE translate OK 1132.1ms
(ServeReplica:ComposedModel pid=16168) INFO 2022-08-09 09:38:44,451 ComposedModel ComposedModel#WaapSf replica.py:482 - HANDLE __call__ OK 5296.8ms
(ServeReplica:SentimentTweet pid=16166) INFO 2022-08-09 09:38:44,545 SentimentTweet SentimentTweet#FBEAev replica.py:482 - HANDLE sentiment OK 83.3ms


...

tweet request... : Cats are better than dogs
tweet response:{"Sentiment": "label:POSITIVE; score=0.9986716508865356", "Translated Text": "Les chats sont meilleurs que les chiens"}


(HTTPProxyActor pid=16160) INFO 2022-08-09 09:38:54,804 http_proxy 127.0.0.1 http_proxy.py:315 - POST /composed 200 412.7ms
(ServeReplica:ComposedModel pid=16168) INFO 2022-08-09 09:38:54,803 ComposedModel ComposedModel#WaapSf replica.py:482 - HANDLE __call__ OK 410.3ms
(ServeReplica:TranslateTweet pid=16165) INFO 2022-08-09 09:38:54,802 TranslateTweet TranslateTweet#jUzajq replica.py:482 - HANDLE translate OK 401.7ms
(ServeReplica:SentimentTweet pid=16166) INFO 2022-08-09 09:38:54,886 SentimentTweet SentimentTweet#FBEAev replica.py:482 - HANDLE sentiment OK 72.3ms


tweet request... : Totally dissastified with the dog. Worst dog ever
tweet response:{"Sentiment": "label:NEGATIVE; score=0.9998103976249695", "Translated Text": "Tr\u00e8s d\u00e9sassass\u00e9 avec le chien, pire chien jamais"}


(HTTPProxyActor pid=16160) INFO 2022-08-09 09:38:55,429 http_proxy 127.0.0.1 http_proxy.py:315 - POST /composed 200 621.0ms
(ServeReplica:ComposedModel pid=16167) INFO 2022-08-09 09:38:55,427 ComposedModel ComposedModel#KRrrri replica.py:482 - HANDLE __call__ OK 617.7ms
(ServeReplica:TranslateTweet pid=16165) INFO 2022-08-09 09:38:55,425 TranslateTweet TranslateTweet#jUzajq replica.py:482 - HANDLE translate OK 608.1ms
(ServeReplica:SentimentTweet pid=16166) INFO 2022-08-09 09:38:55,534 SentimentTweet SentimentTweet#FBEAev replica.py:482 - HANDLE sentiment OK 94.2ms


tweet request... : Briliant dog! Reads my moods like a book. Senses my moods and reacts. What a companinon!
tweet response:{"Sentiment": "label:POSITIVE; score=0.9929038882255554", "Translated Text": "Le chien briliant lise mes humeurs comme un livre, ressent mes humeurs et r\u00e9agit."}


Gracefully shutdown the Ray serve instance.

In [9]:
serve.shutdown()

(ServeController pid=16150) INFO 2022-08-09 09:38:56,506 controller 16150 deployment_state.py:1257 - Removing 2 replicas from deployment 'TranslateTweet'.
(ServeController pid=16150) INFO 2022-08-09 09:38:56,509 controller 16150 deployment_state.py:1257 - Removing 1 replicas from deployment 'SentimentTweet'.
(ServeController pid=16150) INFO 2022-08-09 09:38:56,511 controller 16150 deployment_state.py:1257 - Removing 2 replicas from deployment 'ComposedModel'.
(HTTPProxyActor pid=16160) INFO 2022-08-09 09:38:56,451 http_proxy 127.0.0.1 http_proxy.py:315 - POST /composed 200 1017.1ms
(ServeReplica:TranslateTweet pid=16164) INFO 2022-08-09 09:38:56,449 TranslateTweet TranslateTweet#gIwrMB replica.py:482 - HANDLE translate OK 1005.5ms
(ServeReplica:ComposedModel pid=16168) INFO 2022-08-09 09:38:56,450 ComposedModel ComposedModel#WaapSf replica.py:482 - HANDLE __call__ OK 1014.6ms


### Exercise

1. Add more tweets to `TWEETS` with different sentiments.
2. Check the score (and if you speak and read French, what you think of the translation?)

### Homework

1. Instead of French, use a language transformer of your choice
2. What about Neutral tweets? Try using [vaderSentiment](https://github.com/cjhutto/vaderSentiment)
3. Solution for 2) is [here](https://github.com/anyscale/academy/blob/main/ray-serve/05-Ray-Serve-SentimentAnalysis.ipynb)

### Next

We'll further explore model composition using [Deploymant Graph APIs](https://docs.ray.io/en/latest/serve/deployment-graph.html).

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
➡ [Next notebook](./ex_04_inference_graphs.ipynb) <br>
⬅️ [Previous notebook](./ex_02_ray_serve_fastapi.ipynb) <br>